In [58]:
import pandas as pd
import numpy as np
import re
from re import sub

#for sentiment analyzing
import nltk
import random
from nltk.tokenize import word_tokenize

#for basic preprocessing
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.text import Text
import string, re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [161]:
biden_url = 'Biden Tweets - Biden_Tweets.csv'
biden_df = pd.read_csv(biden_url, error_bad_lines= False).drop(['Unnamed: 1', 'Username'], axis = 1)
biden_df.head(5)

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0
1,Thu Apr 25 16:59:59 +0000 2019,0,pic.twitter.com/PxypFuZzdb,NaN,NaN,0.0
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0


In [162]:
def text_to_word_list(text):

    text = str(text)
    text = text.lower()
    
    if 'pic.twitter' in text:
        text = 'empty'

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ", text)
    text = sub(r"\?", " ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " ", text)
    text = sub(r"\s{2,}", " ", text)
    text = sub(r'http\S+', '', text)

    text = text.split()
    text = list(filter(lambda x: len(x) > 1, text))

    return text  

In [163]:
biden_df["Cleaned_Text"] = biden_df["Text"].apply(lambda x: text_to_word_list(x))
biden_df.head(5)

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment,Cleaned_Text
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0,"[and, we, have, president, trump, to, thank]"
1,Thu Apr 25 16:59:59 +0000 2019,0,pic.twitter.com/PxypFuZzdb,NaN,NaN,0.0,[empty]
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0,"[the, hang, em, high, references, were, less, ..."
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0,"[donald, trump, makes, joe, biden, point, mock..."
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0,"[factually, speaking, that, not, accurate, tru..."


In [164]:
def empty(text):
    is_empty = False
    if text == ['empty']:
        is_empty = True
    return is_empty

In [165]:
def filter_df(df):
    df["is_empty"] = df["Cleaned_Text"].apply(lambda x: empty(x))
    df_filtered = df[df["is_empty"] == False]
    final_df = df_filtered.drop(["is_empty"], axis = 1)
    return final_df

In [166]:
filter_df(biden_df)

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment,Cleaned_Text
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0,"[and, we, have, president, trump, to, thank]"
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0,"[the, hang, em, high, references, were, less, ..."
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0,"[donald, trump, makes, joe, biden, point, mock..."
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0,"[factually, speaking, that, not, accurate, tru..."
5,Thu Apr 25 16:59:57 +0000 2019,0,HAAAAAA!!!,NaN,NaN,0.0,[haaaaaa]
...,...,...,...,...,...,...,...
52774,Sat Apr 11 11:49:39 +0000 2020,0,#NeverBidenNeverTrump,NaN,#NeverBidenNeverTrump,NaN,[neverbidennevertrump]
52776,Sat Apr 11 11:49:37 +0000 2020,0,Yes he have to for some whom haven’t hear it b...,NaN,NaN,NaN,"[yes, he, have, to, for, some, whom, haven, he..."
52777,Sat Apr 11 11:49:36 +0000 2020,0,"No, no... this way Joe, this way........... th...",NaN,NaN,NaN,"[no, no, this, way, joe, this, way, that, the,..."
52779,Sat Apr 11 11:49:36 +0000 2020,0,LA voter- it literally doesn’t matter. You can...,NaN,NaN,NaN,"[la, voter, it, literally, doesn, matter, you,..."
